# Working out database loading

A scratchpad for updating the database, which might eventually become a little more automated.

In [1]:
from spotclient import Client
import models

db = models.Database()
session = models.get_session(create_all=True)

c = Client()
ab = c.allbirds()
latest = ab[0]
# update the database with the latest material
playlist = db.insert_playlist_from_json(session, latest)
print(playlist.name)

JQBX :: Conference of the Birds 2021-06-17


In [4]:
# if that looks right, rebuild the index and commit db changes

db.rebuild_fts(session)
session.commit()


In [3]:
playlist.name

'JQBX :: Conference of the Birds 2021-06-17'

In [ ]:
p = session.query(models.Playlist).filter(models.Playlist.name == 'JQBX :: Conference of the Birds 2021-04-15').scalar()
p

In [ ]:
session.delete(p)
session.commit()

In [ ]:
for i,t in enumerate(p.tracks):
    print(i,t)

In [ ]:
nt = p.tracks[:35] + p.tracks[36:]

In [ ]:
p.tracks = nt

In [ ]:
session.rollback()

In [ ]:
list(p)

In [ ]:
#!cat create_and_populate_track_search.sql | sqlite3 birdnest.db
FTS_INSERT_SQL = '''
insert into track_search (track_id, track, artist, album) 
    select t.track_id,
           t.name track,
           GROUP_CONCAT(a.name,';') artist,
           album.name album
    from 
        playlist p,
        playlist_track pt,
        track t,
        track_artist ta,
        artist a,
        album 
    where t.track_id = ta.track_id 
         and ta.artist_id = a.artist_id
         and p.playlist_id = pt.playlist_id
         and pt.track_id = t.track_id
         and t.album_id = album.album_id
         and p.playlist_id = :playlist_id
    group by t.track_id, t.name'''


In [ ]:
r = session.execute(FTS_INSERT_SQL, params={'playlist_id': 50})

In [ ]:
session.commit()

In [ ]:

def load_all_playlists(session):
    c = Client()
    ab = c.allbirds()
    for p in reversed(ab):
        db.insert_playlist_from_json(session,p)
        print(p['name'])
    fill_in_albums(session) 
        

In [ ]:
track.album.name